In [31]:
!pip install nltk

In [56]:
import pandas as pd
import ast 
from  nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer # to implement bag of words
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
credits = pd.read_csv('tmdb_5000_credits.csv')
movies = pd.read_csv('tmdb_5000_movies.csv')
movies = movies.merge(credits, on = 'title')

In [3]:
# most imp feateres to be selected
# generes
# movie_id
# title
# keywords
# overview
# cast
# crew

In [4]:
movies = movies[['movie_id', 'title','overview','genres','keywords', 'cast','crew']]

In [5]:
# removing melignent data
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [6]:
movies.dropna(inplace = True )

In [7]:
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [8]:
movies.duplicated().sum()

0

In [9]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [10]:
# Writing a function to  convert "genres" features to list
def list_converter(obj):
    list = []
    for i in ast.literal_eval(obj): #this ast function converts the string list to the list
        list.append(i['name'])
    return list   

In [11]:
movies['genres'] = movies['genres'].apply(list_converter)

In [12]:
movies['keywords'] = movies['keywords'].apply(list_converter) #fame for the keywords feild

In [13]:
# Writing a function to  convert "cast" features to list and extract the top actors
def list_converter_cast(obj):
    list = []
    n = 0 
    for i in ast.literal_eval(obj): #this ast function converts the string list to the list
        if n<=3: #need only top 4 acters
            list.append(i['name'])
            n += 1
        else:
            break
    return list  

In [14]:
movies['cast'] = movies['cast'].apply(list_converter_cast)

In [15]:
# Writing a function to  convert "crew" features to list where job is director
def find_director(obj):
    list = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            list.append(i['name'])
    return list
            

In [16]:
movies['crew'] = movies['crew'].apply(find_director)

In [17]:
# making list to overview
movies['overview'] = movies['overview'].apply(lambda x : x.split())

In [18]:
# removing the white spaces in the genres columm
movies['genres'] = movies['genres'].apply(lambda x : [i.replace(" ", "")for i in x] )
movies['cast'] = movies['cast'].apply(lambda x : [i.replace(" ", "")for i in x] )
movies['crew'] = movies['crew'].apply(lambda x : [i.replace(" ", "")for i in x] )
movies['keywords'] = movies['keywords'].apply(lambda x : [i.replace(" ", "")for i in x] )

In [19]:
movies['features'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [20]:
new_data = movies[['movie_id', 'title', 'features']]

In [21]:
new_data.head()

,movie_id,title,features
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [22]:
new_data['features'] = new_data['features'].apply(lambda x : " ".join(x) )

C:\Users\jangi\AppData\Local\Temp\ipykernel_16428\4257917509.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['features'] = new_data['features'].apply(lambda x : " ".join(x) )


In [23]:
# converting to lowercase letters
new_data['features'] = new_data['features'].apply(lambda x : x.lower() )

C:\Users\jangi\AppData\Local\Temp\ipykernel_16428\419174844.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['features'] = new_data['features'].apply(lambda x : x.lower() )


In [24]:
#Now applying the stemming
ps = PorterStemmer()
def stemming(obj):
    list = []
    for i in obj.split():
        list.append(ps.stem(i))
    return " ".join(list)

In [25]:
new_data['features'] = new_data['features'].apply(stemming)
new_data.reset_index(drop = True, inplace = True)

C:\Users\jangi\AppData\Local\Temp\ipykernel_16428\4027937536.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['features'] = new_data['features'].apply(stemming)


In [26]:
#Vectorizing the text 
vectorizer = CountVectorizer( max_features=5000,  stop_words='english')
vectors = vectorizer.fit_transform(new_data['features']).toarray()

In [27]:
vectors.shape

(4806, 5000)

In [28]:
# calculating the cosine similarity for the each vector
cosine_similarity  = cosine_similarity(vectors)

In [54]:
def recommend_movies(movie):
    l = {}
    movie_index = new_data[new_data['title']==movie].index[0]
    distance = cosine_similarity[movie_index]
    movies_list = sorted(list(enumerate(distance)), reverse = True, key = lambda x : x[1])[1:6]
    for i in movies_list:
        movie_title = new_data.iloc[i[0]].title
        movie_id =new_data.iloc[i[0]].movie_id
        l[movie_id] = movie_title
    return l

In [55]:
recommend_movies("Batman Returns")

{155: 'The Dark Knight',
 268: 'Batman',
 49026: 'The Dark Knight Rises',
 2661: 'Batman',
 414: 'Batman Forever'}

In [ ]:
new_data["recommendation"] = new_data["title"].apply(recommend_movies)

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [64]:
movies =new_data[["movie_id","title", "recommendation"]]

In [66]:
movies.to_json("movies.json", orient = "records")